# Введение в Prophet

Для дальнейшей работы понадобится установленная библиотека **FBProphet**: https://facebook.github.io/prophet/.

Для установки откройте терминал анаконды (Anaconda Promt) и запустите следующую команду: `conda install -c conda-forge fbprophet`.

Prophet для визуализации результатов прогнозирования обращается к библиотеке **Plotly**: https://plotly.com/python/, которая позволяет строить интерактивные графики. Можно также установить через conda: `conda install -c plotly plotly=4.6.0` или же через pip. Установка данной библиотеки необязательна.

Подробное описание методов построения прогноза можно найти в работе Forecasting at scale (Facebook Prophet documentation preprint).

Пример построения прогноза рассмотрим на данных о продажах вина в Австралии.

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
wine = pd.read_csv('monthly-australian-wine-sales.csv', parse_dates=['month'], dayfirst=True)

Модель прогнозирования в библиотеке от Facebook имеет название `Prophet`. Основной идеей построения прогноза стало разложение временного ряда на основные составляющие:

$$y(t)=g(t)+s(t)+h(t)+\epsilon_t,$$

где $g(t)$ $-$ функция, описывающая тренд временного ряда, $s(t)$ $-$ компонента, описывающая сезонные колебания, $h(t)$ $-$ компонента, отвечающая за различные праздники и события, которые могут оказывать влияние на целевую переменную $y(t)$, а $\epsilon_t$ представляет собой непрогнозируемую перечисленными компонентами ошибку.

Построим модель прогнозирования для ряда продаж вина.

## 1. Построение модели прогнозирования

In [ ]:
from fbprophet import Prophet

Функция `Prophet` требует, чтобы исходная таблица с данными содержала два столбца: `ds` с моментами времени и `y` с соответствующими значениями целевой переменной. В предыдущей работе при загрузке набора данных `wine` даты использовались в качестве индексов наблюдений. Поскольку теперь для корректной работы функции данные обязательно должны содержать указанные два столбца, используется индекс по умолчанию. Названия столбцов изменим на `ds` и `y`.

In [ ]:
wine.head()

In [ ]:
wine.columns = ['ds', 'y']

Последовательность построения модели схожа с построением `SARIMAX` из statsmodels с единственным отличием: структура модели задается без указания прогнозируемого ряда. Обучение происходит с помощью той же функциии `fit`, где уже указывается набор данных в виде `pd.DataFrame` с указанными выше столбцами.

In [ ]:
model = Prophet(weekly_seasonality=False, daily_seasonality=False)
model.fit(wine)

Для построения прогноза потребуется также передать `pd.DataFrame`, содержащий столбец `ds`. Для создания такой таблицы в Prophet предусмотрена функция `make_future_dataframe`, в которой можно указать горизонт прогнозирования `periods` и частоту наблюдений `freq`. Без указанной частоты будет построен подневный прогноз. Будем также строить прогноз на 3 года вперед.

In [ ]:
future = model.make_future_dataframe(periods=36, freq='MS')

`make_future_dataframe` вернет даты с первого наблюдения до последней даты прогноза.

In [ ]:
future.head()

In [ ]:
future.tail()

Аппроксимацию построенной моделью и прогноз теперь можно получить с помощью функции `predict`, которая вернет `pd.DataFrame` с результатами.

In [ ]:
forecast = model.predict(future)

Приведем оценки модели и прогнозный интервал для последних 12 спрогнозированных значений:

In [ ]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(12)

Для еще большего удобства `Prophet` позволяет сразу отобразить результаты на графике.

In [ ]:
model.plot(forecast)
plt.show()

## 2. Возможности моделирования тренда

Prophet позволяет моделировать затухающий тренд, задав при этом порог насыщения. Такая возможность может пригодиться, например, при прогнозировании числа пассажиров на рейс, где ограничена вместимость воздушного судна, которое назначено на данный рейс. В данном случае порог можно установить равным вместимости судна. Для моделирования затухающего тренда при инициализации модели необходимо указать `growth = 'logistic'`. Установить порог можно, добавив к данным столбец `cap` для ограничения сверху или `floor` для ограничения снизу.

In [ ]:
model_logistic = Prophet(growth='logistic')

## 3. Моделирование сезонности

Для моделирования сезонности в пакете Prophet используется ничто иное как Фурье-преобразования, которые проводятся по формулам:

$$s(t)=\sum_{n=1}^{N}\left(a_n cos \left( \frac{2\pi n t}{P} \right)+b_n sin \left(\frac{2\pi n t}{P}\right) \right).$$

Здесь $N$ $-$ порядок Фурье-преобразования. Для годовой сезонности эмпирическим путем исследователи Facebook определили оптимальный порядок $N=10$, для недельной сезонности $-$ $N=3$.

Сезонные составляющие можно передать в модель при инициализации, а можно добавлять сезонные компоненты к уже определенной модели с указанием периода и порядка преобразования.

In [ ]:
model = Prophet(weekly_seasonality=False, daily_seasonality=False, yearly_seasonality=False)
model.add_seasonality(name='yearly', period=365.25, fourier_order=10)

Таким образом `add_seasonality` позволяет добавить специфические периоды сезонности, характерные для конкретного ряда, например, с периодом сезонности равном 5, если данные собираются исключительно во время рабочей недели.

При наличии мультипликативной сезонности при инициализации модели указывается опция `seasonality_mode='multiplicative'`.

## 4. Добавление праздников

Во временной ряд о продажах вина трудно добавить информацию о праздниках, но по графику заметно, что продажи растут в конце года и падают в начале. Таким образом, можно добавить Новый год в качестве фактора. Список праздников должен быть передан также в виде `pd.DataFrame` с указанием названий праздников в столбце `holiday`, дат праздников `ds`, а также количества дней до `lower_window` и после праздников `upper_window`, в которые также могут наблюдаться изменения спроса.

In [ ]:
new_year = pd.DataFrame({'holiday': 'NewYear', 
                         'ds': pd.to_datetime([str(x)+'-01-01' for x in range(1980, 1997)]),
                         'lower_window': -7, 
                         'upper_window': 7
                        })

In [ ]:
new_year

Список праздников затем передается в модель:

In [ ]:
model_holiday = Prophet(weekly_seasonality=False, daily_seasonality=False, holidays=new_year)

В данном случае добавление праздников не окажет видимого влияния. При рассмотрении подневных данных продаж можно также добавить, например, 23 февраля и 8 марта.

In [ ]:
march_8 = pd.DataFrame({'holiday': 'march_8', 
                         'ds': pd.to_datetime([str(x)+'-03-08' for x in range(2015, 2021)]),
                         'lower_window': -3, 
                         'upper_window': 2
                        })
february_23 = pd.DataFrame({'holiday': 'february_23', 
                         'ds': pd.to_datetime([str(x)+'-02-23' for x in range(2015, 2021)]),
                         'lower_window': -3, 
                         'upper_window': 2
                        })
holidays = pd.concat((march_8, february_23))

In [ ]:
holidays

Официальные праздники разных стран доступны по их двухбуквенному коду в самом пакете.

In [ ]:
model_holidays = Prophet(weekly_seasonality=False, daily_seasonality=False)
model_holidays.add_country_holidays(country_name='AU')
model_holidays.fit(wine)

In [ ]:
model_holidays.train_holiday_names

## 5. Добавление экзогенных факторов

Дополнительные факторы, которые могут улучшить качество прогноза можно добавить с помощью функции `add_regressor`. Исключительно для примера обратим внимание модели на декабрь каждого года.

In [ ]:
def december(ds):
    return 1 if ds.month == 12 else 0

Чтобы добавить дополнительные переменные в модель, их необходимо добавить к матрице с данными `wine` и затем указать в модели названия колонок с дополнительными переменными.

In [ ]:
wine['december'] = wine['ds'].apply(december)

model_december = Prophet(weekly_seasonality=False, daily_seasonality=False)
model_december.add_regressor('december')
model_december.fit(wine)

Для построения прогноза также необходимо добавить экзогенные факторы:

In [ ]:
future_december = model_december.make_future_dataframe(periods=36, freq='MS')
future_december['december'] = future_december['ds'].apply(december)
forecast_december = model_december.predict(future_december)

<div class="alert alert-info">

<h4> Задание (выполнять в отдельном файле)</h4>
<p></p>
Для своего ряда построить прогноз на несколько периодов сезонности (или на 36 значений вперед при отсутствии сезонной составляющей) с применением возможностей библиотеки Prophet. Меняя параметры модели, добавиться наилучшего качества.
</div>

<p></p>
</div>